# Intégration du score 4

Je vais utiliser les données d'immatriculation de véhicules électriques nettoyées par Elsa.

Pour les communes, j'utiliserai le fichier zip downloadé comme Mika.

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
import shutil
import requests
import os

In [33]:
# Lire le fichier des immatriculations
immatriculations = pd.read_csv('data_for_viz/voitures-rechargeables-par-commune-enrichies.csv',
                               dtype={'code_departement': 'str', 'code_region': 'str', 'siren': 'str'})

immatriculations


/tmp/ipykernel_1096/1131568734.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  immatriculations = pd.read_csv('data_for_viz/voitures-rechargeables-par-commune-enrichies.csv',


,codgeo,libgeo,epci,libepci,date_arrete,nb_vp_rechargeables_el,nb_vp_rechargeables_gaz,nb_vp,code_departement,nom_departement,code_region,nom_region,codes_postaux,population,siren
0,35081,CLAYES,243500139,Rennes Métropole,2022-06-30,9,0,821,35,Ille-et-Vilaine,53,Bretagne,['35590'],908.0,213500812
1,35082,COËSMES,243500634,CC Roche aux Fées Communauté,2020-12-31,2,0,1533,35,Ille-et-Vilaine,53,Bretagne,['35134'],1465.0,213500820
2,35082,COËSMES,243500634,CC Roche aux Fées Communauté,2021-03-31,2,0,1534,35,Ille-et-Vilaine,53,Bretagne,['35134'],1465.0,213500820
3,35082,COËSMES,243500634,CC Roche aux Fées Communauté,2021-12-31,4,0,1545,35,Ille-et-Vilaine,53,Bretagne,['35134'],1465.0,213500820
4,35082,COËSMES,243500634,CC Roche aux Fées Communauté,2022-03-31,6,0,1543,35,Ille-et-Vilaine,53,Bretagne,['35134'],1465.0,213500820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316607,23100,JARNAGES,200067544,CC Creuse Confluence,2021-06-30,3,0,527,23,Creuse,75,Nouvelle-Aquitaine,['23140'],439.0,212310007
316608,23100,JARNAGES,200067544,CC Creuse Confluence,2022-03-31,3,0,534,23,Creuse,75,Nouvelle-Aquitaine,['23140'],439.0,212310007
316609,23100,JARNAGES,200067544,CC Creuse Confluence,2022-06-30,5,0,533,23,Creuse,75,Nouvelle-Aquitaine,['23140'],439.0,212310007
316610,23101,JOUILLAT,200034825,CA du Grand Guéret,2020-12-31,0,0,471,23,Creuse,75,Nouvelle-Aquitaine,['23220'],387.0,212310106


In [34]:
immatriculations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316612 entries, 0 to 316611
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   codgeo                   316612 non-null  object 
 1   libgeo                   316612 non-null  object 
 2   epci                     314625 non-null  object 
 3   libepci                  314625 non-null  object 
 4   date_arrete              316612 non-null  object 
 5   nb_vp_rechargeables_el   316612 non-null  int64  
 6   nb_vp_rechargeables_gaz  316612 non-null  int64  
 7   nb_vp                    316612 non-null  int64  
 8   code_departement         314976 non-null  object 
 9   nom_departement          314940 non-null  object 
 10  code_region              314976 non-null  object 
 11  nom_region               314940 non-null  object 
 12  codes_postaux            314976 non-null  object 
 13  population               314976 non-null  float64
 14  sire

Code postaux manquants

In [2]:
# Mika a récupéré les geojson des communes dans car_vizs et get-geo-data. Apparemment c'est lourd.



data_path = 'datas'
file_name2 = 'communes-20220101'
ext2 = '.shp'
url ='https://www.data.gouv.fr/fr/datasets/r/0e117c06-248f-45e5-8945-0e79d9136165'
temp_path = 'temp_unzip'


try: 
    communes = gpd.read_feather(os.path.join(data_path,file_name2 + '.feather'))
except:
    # Zip file from url  
    zip_file = requests.get(url).content
    os.makedirs(temp_path, exist_ok=True)
    with zipfile.ZipFile(io.BytesIO(zip_file)) as archive:
        archive.extractall(temp_path)
    communes = gpd.read_file(os.path.join(temp_path,file_name2 + ext2))   
    shutil.rmtree(temp_path) 
    
    communes.to_feather(os.path.join(data_path,file_name2 + '.feather'))
    

In [39]:
communes['insee'].value_counts()

2B222    1
11258    1
03135    1
03243    1
03078    1
        ..
08288    1
08196    1
08243    1
51054    1
62187    1
Name: insee, Length: 34955, dtype: int64

In [24]:
communes.geometry[0].centroid.x, communes.geometry[0].centroid.y

(9.338150861836196, 42.374292014354154)

In [50]:
immatriculations

,codgeo,libgeo,epci,libepci,date_arrete,nb_vp_rechargeables_el,nb_vp_rechargeables_gaz,nb_vp,code_departement,nom_departement,code_region,nom_region,codes_postaux,population,siren
0,35081,CLAYES,243500139,Rennes Métropole,2022-06-30,9,0,821,35,Ille-et-Vilaine,53,Bretagne,['35590'],908.0,213500812
1,35082,COËSMES,243500634,CC Roche aux Fées Communauté,2020-12-31,2,0,1533,35,Ille-et-Vilaine,53,Bretagne,['35134'],1465.0,213500820
2,35082,COËSMES,243500634,CC Roche aux Fées Communauté,2021-03-31,2,0,1534,35,Ille-et-Vilaine,53,Bretagne,['35134'],1465.0,213500820
3,35082,COËSMES,243500634,CC Roche aux Fées Communauté,2021-12-31,4,0,1545,35,Ille-et-Vilaine,53,Bretagne,['35134'],1465.0,213500820
4,35082,COËSMES,243500634,CC Roche aux Fées Communauté,2022-03-31,6,0,1543,35,Ille-et-Vilaine,53,Bretagne,['35134'],1465.0,213500820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316607,23100,JARNAGES,200067544,CC Creuse Confluence,2021-06-30,3,0,527,23,Creuse,75,Nouvelle-Aquitaine,['23140'],439.0,212310007
316608,23100,JARNAGES,200067544,CC Creuse Confluence,2022-03-31,3,0,534,23,Creuse,75,Nouvelle-Aquitaine,['23140'],439.0,212310007
316609,23100,JARNAGES,200067544,CC Creuse Confluence,2022-06-30,5,0,533,23,Creuse,75,Nouvelle-Aquitaine,['23140'],439.0,212310007
316610,23101,JOUILLAT,200034825,CA du Grand Guéret,2020-12-31,0,0,471,23,Creuse,75,Nouvelle-Aquitaine,['23220'],387.0,212310106


In [55]:
immatriculations[~immatriculations['codes_postaux'].apply(lambda x: isinstance(x,str))]['libgeo'].value_counts()

FORAINS                   891
NON IDENTIFIE             574
LES TROIS LACS              9
SAINT-THIBAUT               9
SAINT-SIMEUX                9
AMBLEVILLE                  9
PREMIÈRES                   9
CRIMOLOIS                   9
SAINT MARTIN                9
SAINT BARTHELEMY            9
LA FAUTE-SUR-MER            9
MUREILS                     9
VILLERS-SOUS-MONTROND       9
CROIXANVEC                  9
VIMARCÉ                     9
SAINT-MARTIN-DE-CONNÉE      9
CHÂTILLON-SUR-LISON         9
ORLIAGUET                   9
CAZOULÈS                    9
SAINT-LOUP-DE-GONOIS        9
LE JARDIN                   9
Name: libgeo, dtype: int64

In [59]:
# drop these rows for now. TODO : Handle NaN codes_postaux
immatriculations_wo_na = immatriculations[immatriculations['codes_postaux'].apply(lambda x: isinstance(x,str))]
immatriculations_wo_na

9      311781
18       2691
27        315
36         72
45         63
54         27
81          9
189         9
144         9
Name: codes_postaux, dtype: int64

In [ ]:
# Several immatriculations have different code postaux
immatriculations_wo_na['codes_postaux'].apply(len).value_counts()

In [60]:
# We could get only last "date_arrete"
immatriculations['date_arrete'].value_counts()

2022-12-31    35183
2022-09-30    35182
2022-06-30    35180
2020-12-31    35178
2021-03-31    35178
2022-03-31    35178
2021-06-30    35178
2021-09-30    35178
2021-12-31    35177
Name: date_arrete, dtype: int64

In [74]:
# We are getting closer to number of communes
immatriculations_recent=immatriculations[immatriculations['date_arrete']=='2022-12-31']
immatriculations_recent

,codgeo,libgeo,epci,libepci,date_arrete,nb_vp_rechargeables_el,nb_vp_rechargeables_gaz,nb_vp,code_departement,nom_departement,code_region,nom_region,codes_postaux,population,siren
10,35086,COMBOURTILLÉ,200072452,CA Fougères Agglomération,2022-12-31,9,0,662,35,Ille-et-Vilaine,53,Bretagne,['35210'],611.0,213500861
13,35087,CORNILLÉ,200039022,CA Vitré Communauté,2022-12-31,6,0,940,35,Ille-et-Vilaine,53,Bretagne,['35500'],964.0,213500879
19,35089,LA COUYÈRE,200070662,CC Bretagne porte de Loire Communauté,2022-12-31,3,0,545,35,Ille-et-Vilaine,53,Bretagne,['35320'],455.0,213500895
29,35091,LE CROUAIS,200038990,CC de Saint-Méen Montauban,2022-12-31,3,0,535,35,Ille-et-Vilaine,53,Bretagne,['35290'],597.0,213500911
32,35092,CUGUEN,243500733,CC Bretagne Romantique,2022-12-31,8,0,991,35,Ille-et-Vilaine,53,Bretagne,['35270'],834.0,213500929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316575,23089,GENOUILLAC,200041556,CC Portes de la Creuse en Marche,2022-12-31,1,0,910,23,Creuse,75,Nouvelle-Aquitaine,['23350'],728.0,212308902
316583,23091,GIOUX,200044014,CC Creuse Grand Sud,2022-12-31,0,0,219,23,Creuse,75,Nouvelle-Aquitaine,['23500'],161.0,212309108
316595,23095,LE GRAND-BOURG,242320000,CC de Bénévent Grand Bourg,2022-12-31,7,0,1363,23,Creuse,75,Nouvelle-Aquitaine,['23240'],1220.0,212309504
316599,23096,GUÉRET,200034825,CA du Grand Guéret,2022-12-31,173,0,13242,23,Creuse,75,Nouvelle-Aquitaine,['23000'],12734.0,212309603


In [75]:
immatriculations_recent[immatriculations_recent['libgeo']=='LE DRENNEC']

,codgeo,libgeo,epci,libepci,date_arrete,nb_vp_rechargeables_el,nb_vp_rechargeables_gaz,nb_vp,code_departement,nom_departement,code_region,nom_region,codes_postaux,population,siren
77795,29047,LE DRENNEC,242900553,CC du Pays des Abers,2022-12-31,18,0,1971,29,Finistère,53,Bretagne,['29860'],1877.0,212900476


In [65]:
# check NaN in there, mostly linked to forains
immatriculations_recent[immatriculations_recent['codes_postaux'].isna()]['libgeo'].value_counts()

FORAINS                   99
NON IDENTIFIE             66
MUREILS                    1
SAINT-THIBAUT              1
SAINT MARTIN               1
VIMARCÉ                    1
LA FAUTE-SUR-MER           1
CROIXANVEC                 1
VILLERS-SOUS-MONTROND      1
CHÂTILLON-SUR-LISON        1
CAZOULÈS                   1
LE JARDIN                  1
SAINT-LOUP-DE-GONOIS       1
SAINT-SIMEUX               1
ORLIAGUET                  1
PREMIÈRES                  1
CRIMOLOIS                  1
SAINT BARTHELEMY           1
LES TROIS LACS             1
SAINT-MARTIN-DE-CONNÉE     1
AMBLEVILLE                 1
Name: libgeo, dtype: int64

In [68]:
immatriculations_recent[immatriculations_recent['libgeo']=='FORAINS']

,codgeo,libgeo,epci,libepci,date_arrete,nb_vp_rechargeables_el,nb_vp_rechargeables_gaz,nb_vp,code_departement,nom_departement,code_region,nom_region,codes_postaux,population,siren
1570,36999,FORAINS,NaN,NaN,2022-12-31,0,0,214,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2396,37999,FORAINS,NaN,NaN,2022-12-31,0,0,371,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5673,43999,FORAINS,NaN,NaN,2022-12-31,0,0,66,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7304,45999,FORAINS,NaN,NaN,2022-12-31,0,0,877,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8276,46999,FORAINS,NaN,NaN,2022-12-31,0,0,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300652,7999,FORAINS,NaN,NaN,2022-12-31,0,0,61,NaN,NaN,NaN,NaN,NaN,NaN,NaN
302017,8999,FORAINS,NaN,NaN,2022-12-31,0,0,308,NaN,NaN,NaN,NaN,NaN,NaN,NaN
305760,5999,FORAINS,NaN,NaN,2022-12-31,0,0,39,NaN,NaN,NaN,NaN,NaN,NaN,NaN
308875,11999,FORAINS,NaN,NaN,2022-12-31,0,0,79,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
immatriculations_recent[immatriculations_recent['libgeo']=='NON IDENTIFIE']

,codgeo,libgeo,epci,libepci,date_arrete,nb_vp_rechargeables_el,nb_vp_rechargeables_gaz,nb_vp,code_departement,nom_departement,code_region,nom_region,codes_postaux,population,siren
16608,34997,NON IDENTIFIE,NaN,NaN,2022-12-31,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20446,67997,NON IDENTIFIE,NaN,NaN,2022-12-31,0,0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24808,52997,NON IDENTIFIE,NaN,NaN,2022-12-31,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34674,59997,NON IDENTIFIE,NaN,NaN,2022-12-31,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36718,60997,NON IDENTIFIE,NaN,NaN,2022-12-31,0,0,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293406,97897,NON IDENTIFIE,NaN,NaN,2022-12-31,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
294963,17997,NON IDENTIFIE,NaN,NaN,2022-12-31,0,0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
297985,1997,NON IDENTIFIE,NaN,NaN,2022-12-31,0,0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
303006,9997,NON IDENTIFIE,NaN,NaN,2022-12-31,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
# Look into communes with several codes postaux
immatriculations_recent_wo_na=immatriculations_recent[~immatriculations_recent['codes_postaux'].isna()]
immatriculations_recent_wo_na

,codgeo,libgeo,epci,libepci,date_arrete,nb_vp_rechargeables_el,nb_vp_rechargeables_gaz,nb_vp,code_departement,nom_departement,code_region,nom_region,codes_postaux,population,siren
10,35086,COMBOURTILLÉ,200072452,CA Fougères Agglomération,2022-12-31,9,0,662,35,Ille-et-Vilaine,53,Bretagne,['35210'],611.0,213500861
13,35087,CORNILLÉ,200039022,CA Vitré Communauté,2022-12-31,6,0,940,35,Ille-et-Vilaine,53,Bretagne,['35500'],964.0,213500879
19,35089,LA COUYÈRE,200070662,CC Bretagne porte de Loire Communauté,2022-12-31,3,0,545,35,Ille-et-Vilaine,53,Bretagne,['35320'],455.0,213500895
29,35091,LE CROUAIS,200038990,CC de Saint-Méen Montauban,2022-12-31,3,0,535,35,Ille-et-Vilaine,53,Bretagne,['35290'],597.0,213500911
32,35092,CUGUEN,243500733,CC Bretagne Romantique,2022-12-31,8,0,991,35,Ille-et-Vilaine,53,Bretagne,['35270'],834.0,213500929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316575,23089,GENOUILLAC,200041556,CC Portes de la Creuse en Marche,2022-12-31,1,0,910,23,Creuse,75,Nouvelle-Aquitaine,['23350'],728.0,212308902
316583,23091,GIOUX,200044014,CC Creuse Grand Sud,2022-12-31,0,0,219,23,Creuse,75,Nouvelle-Aquitaine,['23500'],161.0,212309108
316595,23095,LE GRAND-BOURG,242320000,CC de Bénévent Grand Bourg,2022-12-31,7,0,1363,23,Creuse,75,Nouvelle-Aquitaine,['23240'],1220.0,212309504
316599,23096,GUÉRET,200034825,CA du Grand Guéret,2022-12-31,173,0,13242,23,Creuse,75,Nouvelle-Aquitaine,['23000'],12734.0,212309603


In [83]:
# Several communes have different code postaux
immatriculations_recent_wo_na['codes_postaux'].apply(len).value_counts()

9      34644
18       299
27        35
36         8
45         7
54         3
81         1
144        1
189        1
Name: codes_postaux, dtype: int64

In [85]:
# This is the case of big cities for instance. We may want to explode these rows in several rows
immatriculations_recent_wo_na[immatriculations_recent_wo_na['codes_postaux'].apply(len)>9]

,codgeo,libgeo,epci,libepci,date_arrete,nb_vp_rechargeables_el,nb_vp_rechargeables_gaz,nb_vp,code_departement,nom_departement,code_region,nom_region,codes_postaux,population,siren
254,35163,LUITRÉ-DOMPIERRE,200072452,CA Fougères Agglomération,2022-12-31,31,0,1963,35,Ille-et-Vilaine,53,Bretagne,"['35133', '35210']",1823.0,200084002
485,35238,RENNES,243500139,Rennes Métropole,2022-12-31,2181,7,157301,35,Ille-et-Vilaine,53,Bretagne,"['35000', '35200', '35700']",220488.0,213502388
1637,37021,BEAUMONT-LOUESTAULT,200073237,CC de Gâtine et Choisilles - Pays de Racan,2022-12-31,30,0,1700,37,Indre-et-Loire,24,Centre-Val de Loire,"['37360', '37370']",1719.0,200071124
2999,38225,AUTRANS-MÉAUDRE EN VERCORS,243801024,CC du Massif du Vercors,2022-12-31,60,0,3266,38,Isère,84,Auvergne-Rhône-Alpes,"['38112', '38880']",3039.0,200056224
4557,42218,SAINT-ÉTIENNE,244200770,Saint-Etienne Métropole,2022-12-31,1888,33,149003,42,Loire,84,Auvergne-Rhône-Alpes,"['42000', '42100', '42230']",173821.0,214202186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311315,14591,AURE SUR MER,200066801,CC Isigny-Omaha Intercom,2022-12-31,7,0,821,14,Calvados,28,Normandie,"['14520', '14710']",732.0,200073377
312045,15142,NEUVÉGLISE-SUR-TRUYÈRE,200066660,CC Saint-Flour Communauté,2022-12-31,9,0,2141,15,Cantal,84,Auvergne-Rhône-Alpes,"['15100', '15260']",1735.0,200065191
312388,15267,YTRAC,241500230,CA du Bassin d'Aurillac,2022-12-31,59,0,4461,15,Cantal,84,Auvergne-Rhône-Alpes,"['15000', '15130']",4335.0,211502679
312688,16110,COURCÔME,200043016,CC Val de Charente,2022-12-31,5,0,904,16,Charente,75,Nouvelle-Aquitaine,"['16240', '16700']",777.0,200086429


In [94]:
# Example of Paris
immatriculations_recent_wo_na[
    (immatriculations_recent_wo_na['libgeo'].str.contains('PARIS'))
]


,codgeo,libgeo,epci,libepci,date_arrete,nb_vp_rechargeables_el,nb_vp_rechargeables_gaz,nb_vp,code_departement,nom_departement,code_region,nom_region,codes_postaux,population,siren
28440,71343,PARIS-L'HÔPITAL,200006682,"CA Beaune, Côte et Sud - Communauté Beaune-Cha...",2022-12-31,5,0,299,71,Saône-et-Loire,27,Bourgogne-Franche-Comté,['71150'],318.0,217103431
70398,38485,SEYSSINET-PARISET,200040715,Grenoble-Alpes-Métropole,2022-12-31,191,2,11663,38,Isère,84,Auvergne-Rhône-Alpes,['38170'],11891.0,213804859
81846,77514,VILLEPARISIS,200055655,CA Roissy Pays de France,2022-12-31,217,0,24766,77,Seine-et-Marne,11,Île-de-France,['77270'],26580.0,217705144
88215,75104,PARIS 4E ARRONDISSEMENT,NaN,NaN,2022-12-31,530,0,12816,75,Paris,11,Île-de-France,['75004'],29131.0,NaN
88220,75106,PARIS 6E ARRONDISSEMENT,NaN,NaN,2022-12-31,894,0,21293,75,Paris,11,Île-de-France,['75006'],40303.0,NaN
88244,75116,PARIS 16E ARRONDISSEMENT,NaN,NaN,2022-12-31,5983,4,114864,75,Paris,11,Île-de-France,"['75016', '75116']",165523.0,NaN
88248,75117,PARIS 17E ARRONDISSEMENT,NaN,NaN,2022-12-31,3984,5,93139,75,Paris,11,Île-de-France,['75017'],166543.0,NaN
88251,75118,PARIS 18E ARRONDISSEMENT,NaN,NaN,2022-12-31,1037,9,71873,75,Paris,11,Île-de-France,['75018'],192468.0,NaN
137377,75101,PARIS 1ER ARRONDISSEMENT,NaN,NaN,2022-12-31,1284,2,13287,75,Paris,11,Île-de-France,['75001'],15917.0,NaN
137381,75103,PARIS 3E ARRONDISSEMENT,NaN,NaN,2022-12-31,574,0,13074,75,Paris,11,Île-de-France,['75003'],34025.0,NaN


In [96]:
communes[communes['nom_upper'].str.contains('PARIS')]

,insee,nom,wikipedia,surf_ha,geometry,nom_upper
1207,38485,Seyssinet-Pariset,fr:Seyssinet-Pariset,1078.0,"POLYGON ((5.63864 45.15350, 5.63886 45.15434, ...",SEYSSINET-PARISET
6812,39189,Damparis,fr:Damparis,895.0,"POLYGON ((5.36102 47.07591, 5.36207 47.07670, ...",DAMPARIS
10410,71343,Paris-l'Hôpital,fr:Paris-l'Hôpital,275.0,"POLYGON ((4.62839 46.90759, 4.62841 46.90787, ...",PARIS-L'HÔPITAL
25829,95176,Cormeilles-en-Parisis,fr:Cormeilles-en-Parisis,844.0,"POLYGON ((2.17335 48.95349, 2.17398 48.95391, ...",CORMEILLES-EN-PARISIS
26071,81202,Parisot,fr:Parisot (Tarn),2871.0,"POLYGON ((1.78504 43.80676, 1.78506 43.80678, ...",PARISOT
26261,82137,Parisot,fr:Parisot (Tarn-et-Garonne),2822.0,"POLYGON ((1.81972 44.28128, 1.81976 44.28131, ...",PARISOT
27694,62826,Le Touquet-Paris-Plage,fr:Le Touquet-Paris-Plage,1528.0,"POLYGON ((1.57614 50.48773, 1.57616 50.48790, ...",LE TOUQUET-PARIS-PLAGE
32143,77514,Villeparisis,fr:Villeparisis,842.0,"POLYGON ((2.59139 48.95008, 2.59142 48.95030, ...",VILLEPARISIS
32282,75056,Paris,fr:Paris,10539.0,"POLYGON ((2.22412 48.85420, 2.22416 48.85461, ...",PARIS
33849,95241,Fontenay-en-Parisis,fr:Fontenay-en-Parisis,1102.0,"POLYGON ((2.41128 49.05330, 2.41333 49.05611, ...",FONTENAY-EN-PARISIS


**All arrondissements of Paris need to be summed to Paris with codgeo 75056. Probably need to do this for all communes with multiple codes-postaux**

In [ ]:
immatriculations_with_communes = immatriculations.merge(communes, left_on='code_insee_commune', right_on='code_commune_INSEE', how='left')
immatriculations_with_communes

In [ ]:
def score_4_commune(
    all_cars_df:pd.DataFrame,
    commune:str,
):
    """Compute score 4 for the commune."""
    x_commune, y_commune = all_cars_df[all_cars_df['commune']==commune].iloc[0][['x','y']].values
    # Get cars in 20 km around the commune
    around_cars=all_cars_df[np.sqrt((all_cars_df['x']-x_commune)**2+(all_cars_df['y']-y_commune)**2)<20]
    score = sum(
        around_cars.groupby('commune')
            .agg(len) # get number of cars in the commune
            / (1+gamma*around_cars.groupby('commune').agg(lambda x,y : ((x-x_commune)**2+(y-y_commune)**2)))
    )
    return score